# The `apssh` menagerie

* `SshJob`: specialized `asynciojobs` Job

* `SshNode`: object needed to build an `SshJob`

# `apssh` extends `asynciojobs`

![](sched+jobs.png)

![](sched+sshjobs.png)

My first ping

In [ ]:
from asynciojobs import Scheduler
from apssh import SshNode, SshJob

gateway = SshNode(
    'faraday.inria.fr',
    username='inria_school'
)

In [ ]:
s = Scheduler()

simplest_ping = SshJob(
    node = gateway,
    command = 'ping -c 4 google.com',
    scheduler = s,
)

In [ ]:
s.graph()

À ce stade on n'a pas créé de connexion

In [ ]:
s.run()

# digression : load image on all nodes 

We can do this manually of course :

```
ssh inria_school@faraday.inria.fr

rhubarbe load --all --image ubuntu --curses

rhubarbe load -a -i ubuntu -c

```

Then to wait for the nodes to reboot
```
rhubarbe wait -a -c

```

# multi-hop ssh

Now that the nodes are turned on:

* how to run stuff on nodes through the gateway
* since they have no public IP

For that purpose:

* we create a `SshNode`
* with a `gateway` attribute
* that itself is a `SshNode`


In [ ]:
n10 = SshNode(
    'fit10',
    username='root',
    gateway=gateway,
)

In [ ]:
s = Scheduler()

SshJob(
    node = n10,
    command = "hostname",
    scheduler = s,
)

In [ ]:
s.run()

# `apssh` menagerie - commands

Commands can be defined in many different - equivalent - forms

    SshJob(command = "hostname", ...)
    SshJob(commands = "hostname", ...)
    SshJob(commands = ["hostname"], ...)

In [ ]:
from apssh import Run

    SshJob(commands = [Run("hostname")], ...)

It is recommended to use `Run`

In [ ]:
s = Scheduler()

SshJob(
    node = n10,
    commands = [Run("hostname")],
    scheduler = s,
)

In [ ]:
s.run()

# Run local scripts remotely

It can be helpful to have local scripts - e.g. in your git repo - that you want to run remotely:

In [ ]:
# let us assume we have a local script
!cat node-tools.sh

In [ ]:
from apssh import RunScript
s = Scheduler()

SshJob(
    node = n10,
    commands = [RunScript("node-tools.sh",
                          "init-ad-hoc-network",
                          "intel",
                          "foobar",
                         )],
    scheduler = s,
)

s.run()

# Run local scripts remotely - 2 

The `RunString` class is similar to `RunScript`
* but uses a python string as the source
* instead of a local file

# File transfers

The `Push` and `Pull` classes can be used as commands.

In [ ]:
from apssh import Pull

s = Scheduler()

SshJob(
    node = n10,
    commands = [ 
        Run("echo $(hostname) at $(date) > HOSTNAME"),
        Pull("HOSTNAME", "fit10-hostname"),
    ],
    scheduler = s,
)

s.run()

In [ ]:
!cat fit10-hostname

# Formatting

Each `SshNode` has a formatter instance attached

* this allows to customize all outputs (mention hostname or not, add timestamp, etc..)
* also useful to capture output (like in shell's `$(ssh host command)`)
  * see [CaptureFormatter](http://apssh.readthedocs.io/en/latest/API.html?highlight=capture#apssh.formatters.CaptureFormatter)_

# Imaging 

In [ ]:
# programmatively load images on nodes

nodes = 10, 12

s = Scheduler()
SshJob( 
    node = gateway, 
    commands = [
        Run("rhubarbe load -i fedora", *nodes),
        Run("rhubarbe wait", *nodes),
    ],
    scheduler = s,
)

s.run()
